# Hussaini lab data (Axona) conversion to NWB and TINT

Here, I am going to test code for doing said conversions.

### NWB

To convert to NWB we will need a separate Interface class (e.g. AxonaRecordingExtractorInterface) for each data type / extractor. Those are then combined with an NWBConverter class for the Lab (e.g. HussainiLabNWBConverter), which exports to NWB. 

Adapted from https://github.com/catalystneuro/movshon-lab-to-nwb/blob/main/tutorials/blackrock_nwb_conversion_detailed.ipynb

In [24]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [25]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [26]:
!ls ../nwb-conversion-tools/nwb_conversion_tools -l

total 76
-rwxrwxrwx 1 steburg steburg    69 Apr  5 11:28 __init__.py
drwxrwxrwx 1 steburg steburg  4096 Apr  8 17:40 __pycache__
-rwxrwxrwx 1 steburg steburg  1404 Apr  5 11:28 auto_qc.py
-rwxrwxrwx 1 steburg steburg  1338 Apr  5 11:28 basedatainterface.py
-rwxrwxrwx 1 steburg steburg  2134 Apr  5 11:28 baseimagingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2775 Apr  5 11:28 baselfpextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4629 Apr  5 11:28 baserecordingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2363 Apr  5 11:28 basesegmentationextractorinterface.py
-rwxrwxrwx 1 steburg steburg  3829 Apr  5 11:28 basesortingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  5003 Apr  8 17:38 conversion_tools.py
drwxrwxrwx 1 steburg steburg  4096 Apr  8 17:38 datainterfaces
-rwxrwxrwx 1 steburg steburg  5100 Apr  5 11:28 json_schema_utils.py
-rwxrwxrwx 1 steburg steburg 10154 Apr  5 11:28 metafile.schema.json
-rwxrwxrwx 1 steburg steburg  6665 Apr  5 11:28 nwbconverter.py
dr

In [27]:
# Import modules

import random
import string
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np
import re
import datetime

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface,  BaseDataInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from nwb_conversion_tools.json_schema_utils import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools.datainterfaces.interface_utils.brpylib import NsxFile
from nwb_conversion_tools import SpikeGLXRecordingInterface

In [28]:
# from basedatainterface.py
base_schema = get_base_schema(
    id_='metadata.schema.json',
    root=True,
    title='Metadata',
    description='Schema for the metadata',
    version="0.1.0",
)

In [29]:
base_schema

{'required': [],
 'properties': {},
 'type': 'object',
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'metadata.schema.json',
 'title': 'Metadata',
 'description': 'Schema for the metadata',
 'version': '0.1.0'}

In [30]:
glx = SpikeGLXRecordingInterface

In [31]:
glx.get_source_schema()

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [32]:
import inspect

inspect.signature(glx.__init__).parameters

mappingproxy({'self': <Parameter "self">,
              'file_path': <Parameter "file_path: Union[str, pathlib.Path, NoneType]">,
              'stub_test': <Parameter "stub_test: Union[bool, NoneType] = False">})

In [33]:
!ls ../spikeextractors/spikeextractors -l

total 224
-rwxrwxrwx 1 steburg steburg    86 Mar 24 10:00 MANIFEST.in
-rwxrwxrwx 1 steburg steburg  1265 Mar 19 08:48 __init__.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 09:53 __pycache__
-rwxrwxrwx 1 steburg steburg 22834 Mar 19 08:52 baseextractor.py
-rwxrwxrwx 1 steburg steburg  9269 Mar 19 08:52 cacheextractors.py
drwxrwxrwx 1 steburg steburg  4096 Mar 24 10:00 example_datasets
-rwxrwxrwx 1 steburg steburg   105 Mar 19 08:48 exceptions.py
-rwxrwxrwx 1 steburg steburg 36890 Mar 19 08:52 extraction_tools.py
-rwxrwxrwx 1 steburg steburg  5952 Apr  1 17:10 extractorlist.py
drwxrwxrwx 1 steburg steburg  4096 Mar 22 09:47 extractors
-rwxrwxrwx 1 steburg steburg  6494 Mar 19 08:52 multirecordingchannelextractor.py
-rwxrwxrwx 1 steburg steburg  8390 Mar 19 08:52 multirecordingtimeextractor.py
-rwxrwxrwx 1 steburg steburg  5458 Mar 19 08:52 multisortingextractor.py
-rwxrwxrwx 1 steburg steburg 41357 Mar 19 08:52 recordingextractor.py
-rwxrwxrwx 1 steburg steburg 30642 Mar 19 08:52 sorting

In [34]:
from spikeextractors.extractors.neoextractors import AxonaRecordingExtractor

In [35]:
are = AxonaRecordingExtractor(filename=filename)
print('Number of channels:', are.get_num_channels())
print('Channel groups:', are.get_channel_groups())

Number of channels: 16
Channel groups: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Parse .set file for metadata to include in metadata.

In [37]:
def parse_generic_header(filename, params):
    """
    Given a binary file with phrases and line breaks, enters the
    first word of a phrase as dictionary key and the following
    string (without linebreaks) as value. Returns the dictionary.
    
    INPUT
    filename (str): .set file path and name.
    params (list or set): parameter names to search for. 
    
    OUTPUT
    header (dict): dictionary with keys being the parameters that
                   were found & values being strings of the data.
                   
    EXAMPLE
    parse_generic_header('myset_file.set', ['experimenter', 'trial_time'])
    """
    header = {}
    params = set(params)
    with open(filename, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                break
            line = bin_line.decode('cp1252').replace('\r\n', '').replace('\r', '').strip()
            parts = line.split(' ')
            key = parts[0]
            if key in params:
                header[key] = ' '.join(parts[1:])
            
    return header

In [15]:
params_of_interest = [
    'experimenter', 
    'comments',
    'duration', 
    'sw_version',
    'tracker_version',
    'stim_version',
    'audio_version'
]

In [16]:
parse_generic_header(set_file, params_of_interest)

{'experimenter': 'Abid',
 'comments': '',
 'duration': '600.00625',
 'sw_version': '1.2.2.16',
 'tracker_version': '0',
 'stim_version': '1',
 'audio_version': '0'}

In [17]:
def read_iso_datetime(set_file):
    """ 
    Creates datetime object (y, m, d, h, m, s) from .set file header 
    """
    with open(set_file, 'r', encoding='cp1252') as f:
        for line in f:
            if line.startswith('trial_date'):
                date_string = re.findall(r'\d+\s\w+\s\d{4}$', line)[0]
            if line.startswith('trial_time'):
                time_string = line[len('trial_time')+1::].replace('\n', '')

    return datetime.datetime.strptime(date_string + ', ' + time_string, \
        "%d %b %Y, %H:%M:%S").isoformat()

In [32]:
class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['filename'],
            'properties': {
                'filename': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
       
    def __init__(self, **source_data):
        super().__init__(**source_data)
        
    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""        
                
        # Extract information for specific parameters from .set file
        params_of_interest = [
            'experimenter', 
            'comments',
            'duration', 
            'sw_version',
            'tracker_version',
            'stim_version',
            'audio_version'
        ]
        set_file = self.source_data['filename']+'.set'
        par = parse_generic_header(set_file, params_of_interest)
        
        # Extract information from AxonaRecordingExtractor
        elec_group_names = self.recording_extractor.get_channel_groups()
        unique_elec_group_names = set(elec_group_names)
        
        # Add available metadata
        # NOTE that this interface is meant to be used within an NWBconverter, 
        # which contains a much larger metadata_schema. As such a datainterface
        # metadata by itself does not necessarily validate with its own schema!
        metadata = super().get_metadata()
        metadata['NWBFile'] = dict(
            session_start_time=read_iso_datetime(set_file),
            session_description=par['comments'],
            identifier=''.join(random.choices(string.ascii_uppercase + string.digits, k=12)),
            #session_duration=par['duration']+'s',
            #tracker_version=par['tracker_version'],
            #stim_version=par['stim_version'],
            #audio_version=par['audio_version']
            experimenter=[par['experimenter']]
        )
        
        metadata['Ecephys'] = dict(
            Device=dict(
                    name="Axona",
                    description="Axona DacqUSB, sw_version={}".format(par['sw_version']),
                    manufacturer="Axona"
                ),
            #ElectrodeGroup=dict(
            #    name='Group0',
            #    location='',
            #    device='',
            #    description="Group0 - all electrodes grouped together.",
            #),
            ElectrodeGroup=[
                dict(
                    name=f'Group{group_name}',
                    location='',
                    device='',
                    description=f"Group {group_name} electrodes.",
                )
                for group_name in unique_elec_group_names
            ],
            # Electrodes is not in the metadata_schema by default... 
            #Electrodes=[
            #    dict(
            #        name='group_name',
            #        description="The name of the ElectrodeGroup this electrode is a part of.",
            #        data=[f"Group{x}" for x in elec_group_names]
            #    )
            #],
            ElectricalSeries=dict(
                name='ElectricalSeries',
                description="Raw acquisition traces."
            )
        )
  
        return metadata

In [33]:
RXI = AxonaRecordingExtractorInterface(filename=filename)

In [34]:
RXI.get_metadata()

{'NWBFile': {'session_start_time': '2020-10-04T11:07:07',
  'session_description': '',
  'identifier': 'ZZ2FYRP0AEVN',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': {'name': 'Axona',
   'description': 'Axona DacqUSB, sw_version=1.2.2.16',
   'manufacturer': 'Axona'},
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': '',
    'description': 'Group 0 electrodes.'}],
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [35]:
base_metadata_schema = RXI.get_metadata_schema()

In [36]:
import json
print(json.dumps(base_metadata_schema, indent=2))

{
  "required": [],
  "properties": {
    "Ecephys": {
      "required": [
        "Device",
        "ElectrodeGroup",
        "ElectricalSeries"
      ],
      "properties": {
        "Device": {
          "required": [
            "name"
          ],
          "properties": {
            "name": {
              "description": "the name of this device",
              "type": "string",
              "default": "Axona"
            },
            "description": {
              "description": "Description of the device (e.g., model, firmware version, processing software version, etc.)",
              "type": "string",
              "default": "Axona DacqUSB, sw_version=1.2.2.16"
            },
            "manufacturer": {
              "description": "the name of the manufacturer of this device",
              "type": "string",
              "default": "Axona"
            }
          },
          "type": "object",
          "additionalProperties": false,
          "tag": "pynwb.device.De

In [37]:
# The metadata_schema does not validate with the metadata, bc
# the full metadata_schema is only inherited when defining the
# nwbconverter (see below).

from jsonschema import validate, ValidationError

try:
    validate(
        instance=RXI.get_metadata(),
        schema=RXI.get_metadata_schema()
    )
    print('validation SUCCESS')
except ValidationError:
    print('validation FAILED')

validation FAILED


In [38]:
# So let's create an AxonaNWBconverter

# Here we will also import all necessary datainterfaces
from nwb_conversion_tools import (
    NWBConverter
)


# A separate class for each datatype
class HussainiRawNWBConverter(NWBConverter):
    
    '''
    # If I want to be able to add arbitrary additional / optional properties
    def get_metadata_schema(cls):
        metadata_schema = super().get_metadata_schema()
        metadata_schema['properties']['NWBFile']['additionalProperties'] = True
        return metadata_schema
    '''
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface, 
        # AxonaSortingExtractorInterface=AxonaSortingExtractorInterface
    )

In [39]:
# Get source_schema from converter

source_schema = HussainiRawNWBConverter.get_source_schema()
print(json.dumps(source_schema['properties'], indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "required": [
      "filename"
    ],
    "properties": {
      "filename": {
        "type": "string",
        "format": "file",
        "description": "Path to Axona files."
      }
    },
    "type": "object",
    "additionalProperties": true
  }
}


In [40]:
# Define source data (required for instantiating converter)

# TODO add sortingextractor
source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)

print(json.dumps(source_data, indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s"
  }
}


In [41]:
# Instantiate Hussaini-lab converter

converter = HussainiRawNWBConverter(source_data=source_data)

In [42]:
# Get metadata_schema from converter

metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))

{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": ""
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "MG4NPH7R77QN"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "2020-10-04T11:07:07"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment",
        "default": [
          "Abid"
        ]
      },
      "experiment_description": 

In [43]:
metadata = converter.get_metadata()
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': '8ASURR384BLY',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': {'name': 'Axona',
   'description': 'Axona DacqUSB, sw_version=1.2.2.16',
   'manufacturer': 'Axona'},
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': '',
    'description': 'Group 0 electrodes.'}],
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [44]:
metadata['Ecephys']

{'Device': {'name': 'Axona',
  'description': 'Axona DacqUSB, sw_version=1.2.2.16',
  'manufacturer': 'Axona'},
 'ElectrodeGroup': [{'name': 'Group0',
   'location': '',
   'device': '',
   'description': 'Group 0 electrodes.'}],
 'ElectricalSeries': {'name': 'ElectricalSeries',
  'description': 'Raw acquisition traces.'}}

In [45]:
# Validate metadata with metadata_schema from nwbconverter

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

ValidationError: [{'name': 'Group0', 'location': '', 'device': '', 'description': 'Group 0 electrodes.'}] is not of type 'object'

Failed validating 'type' in schema['properties']['Ecephys']['properties']['ElectrodeGroup']:
    {'additionalProperties': False,
     'properties': {'description': {'description': 'description of this '
                                                   'electrode group',
                                    'type': 'string'},
                    'device': {'description': 'the device that was used to '
                                              'record from this electrode '
                                              'group',
                               'target': 'pynwb.device.Device',
                               'type': 'string'},
                    'location': {'description': 'description of location '
                                                'of this electrode group',
                                 'type': 'string'},
                 

## Test axonadatainterface from nwb_conversion_tools

In [1]:
import json
from jsonschema import validate

In [2]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [3]:
#from nwb_conversion_tools import axonadatainterface

In [4]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface
)

In [5]:
import sys
sys.path.append('/mnt/d/spikeinterface/hussaini-lab-to-nwb/hussaini_lab_to_nwb')

In [6]:
from hussaininwbconverter import HussainiNWBConverter

In [7]:
# Get source_schema from converter

source_schema = HussainiNWBConverter.get_source_schema()
print(json.dumps(source_schema['properties'], indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "required": [
      "filename"
    ],
    "properties": {
      "filename": {
        "type": "string",
        "format": "file",
        "description": "Path to Axona files."
      }
    },
    "type": "object",
    "additionalProperties": true
  }
}


In [8]:
source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)
print(json.dumps(source_data, indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s"
  }
}


In [9]:
# Validate source_data with source_schema

validate(
    instance=source_data,
    schema=source_schema
)

In [10]:
# Instantiate Hussaini-lab converter

converter = HussainiNWBConverter(source_data=source_data)

In [11]:
# Get metadata_schema from converter

metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))


metadata = converter.get_metadata()
metadata

{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": ""
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "BWBFMMXHFCPL"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "2020-10-04T11:07:07"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment",
        "default": [
          "Abid"
        ]
      },
      "experiment_description": 

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': '7IVBN48A4SI3',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': {'name': 'Axona',
   'description': 'Axona DacqUSB, sw_version=1.2.2.16',
   'manufacturer': 'Axona'},
  'ElectrodeGroup': {'name': 'Group0',
   'location': '',
   'device': '',
   'description': 'Group0 - all electrodes grouped together.'},
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [12]:
# Validate metadata with metadata_schema from nwbconverter

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

In [13]:
# Test pytest function test_interface_schemas() for this specific interface

from jsonschema import Draft7Validator

In [14]:
data_interface = AxonaRecordingExtractorInterface(filename=filename)

schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

In [15]:
data_interface

In [18]:
import nwb_conversion_tools

In [19]:
# Try to see for which data_interface pytest crashes
# Only for tutorialinterfaces! But it works fine for AxonaRecordingExtractorInterface!

interfaces_that_crash = [nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialRecordingInterface,
                         nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialSortingInterface]

from nwb_conversion_tools import interface_list

for data_interface in interface_list:
    
    if not data_interface in interfaces_that_crash:
    
        # check validity of source schema
        schema = data_interface.get_source_schema()
        Draft7Validator.check_schema(schema)

        # check validity of conversion options schema
        schema = data_interface.get_conversion_options_schema()
        Draft7Validator.check_schema(schema)

In [22]:
interfaces_that_crash = [nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialRecordingInterface,
                         nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialSortingInterface]

data_interface = interfaces_that_crash[0]
        
# check validity of source schema
schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

ValueError: No valid arguments were found in the json type mapping!

In [23]:
data_interface = interfaces_that_crash[1]
        
# check validity of source schema
schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

ValueError: No valid arguments were found in the json type mapping!